In [ ]:
# Aggiungi questo blocco all'inizio della tua cella di Jupyter Notebook
import os
import logging
import warnings

# Metodo 1: Imposta il livello di log di TensorFlow (il più aggressivo)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Metodo 2: Imposta il livello di log per il modulo 'absl'
logging.getLogger('absl').setLevel(logging.ERROR)

# Metodo 3: Filtra specifici messaggi di warning di Python
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow', message="Input of GeneratorDatasetOp")
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow', message="Local rendezvous is aborting with status")


import numpy as np
import random
import sys
import os
import hashlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time
from tqdm import tqdm


# === Hardware settings ===
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/leonardo/prod/spack/06/install/0.22/linux-rhel8-icelake/gcc-8.5.0/cuda-12.2.0-o6rr2unwsp4e4av6ukobro6plj7ceeos'

# === Parameters ===
BETA = 10
M = 4
X0_INITIAL = 0.9
SNR_TEST_RANGE_DB = np.arange(0, 21, 1)
SNR_TRAIN_RANGE_DB = [0, 20]
NUM_SYMBOLS_TRAIN = 60000
NUM_SYMBOLS_VALIDATION = 20000
NUM_SYMBOLS_TEST_PER_SNR = 500000
CHANNEL_TYPE = 'Rayleigh'
L_FADING = 1

MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED)
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# === Strategy ===
try:
    strategy = tf.distribute.MirroredStrategy()
    print(f'Numero di dispositivi in uso dalla strategia: {strategy.num_replicas_in_sync}')
except RuntimeError as e:
    print(f"Errore nella configurazione della strategia di distribuzione: {e}")
    strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")

# === Signal generation (vectorized) ===
def _generate_test_batch_signals(num_symbols, snr_db, beta, M, channel_type='AWGN', L=1,seed =42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    y_labels = np.random.randint(0, 2, size=num_symbols)
    x0_values = np.random.rand(num_symbols)
    chaotic_sequences = np.zeros((num_symbols, beta))
    bernoulli_indices = np.where(y_labels == 0)[0]
    logistic_indices = np.where(y_labels == 1)[0]

    if len(bernoulli_indices) > 0:
        x_b = x0_values[bernoulli_indices]
        seq_b = np.zeros((len(x_b), beta))
        for i in range(beta):
            x_b = (2 * x_b) % 1
            seq_b[:, i] = x_b
        chaotic_sequences[bernoulli_indices] = seq_b

    if len(logistic_indices) > 0:
        x_l = x0_values[logistic_indices]
        seq_l = np.zeros((len(x_l), beta))
        rho = 3.6
        for i in range(beta):
            x_l = rho * x_l * (1 - x_l)
            seq_l[:, i] = x_l
        chaotic_sequences[logistic_indices] = seq_l

    modulated_signals = np.tile(chaotic_sequences[:, np.newaxis, :], (1, M, 1))

    if channel_type.lower() == 'awgn':
        signal_power = np.mean(np.abs(modulated_signals)**2, axis=(1, 2), keepdims=True)
        snr_linear = 10**(snr_db / 10)
        noise_power = signal_power / snr_linear
        noise_amplitude = np.sqrt(noise_power / 2)
        noise_real = np.random.randn(*modulated_signals.shape)
        noise_imag = np.random.randn(*modulated_signals.shape)
        noise = noise_amplitude * (noise_real + 1j * noise_imag)
        received_signals = modulated_signals + noise
    elif channel_type.lower() == 'rayleigh':
        received_signals = np.zeros_like(modulated_signals, dtype=np.complex64)
        for _ in range(L):
            h = (np.random.randn(num_symbols, M, 1) + 1j * np.random.randn(num_symbols, M, 1)) / np.sqrt(2)
            received_signals += h * modulated_signals
        original_power = np.mean(np.abs(modulated_signals)**2, axis=(1, 2), keepdims=True)
        current_power = np.mean(np.abs(received_signals)**2, axis=(1, 2), keepdims=True)
        received_signals *= np.sqrt(original_power / current_power)
        signal_power = np.mean(np.abs(received_signals)**2, axis=(1, 2), keepdims=True)
        snr_linear = 10**(snr_db / 10)
        noise_power = signal_power / snr_linear
        noise_amplitude = np.sqrt(noise_power / 2)
        noise_real = np.random.randn(*received_signals.shape)
        noise_imag = np.random.randn(*received_signals.shape)
        noise = noise_amplitude * (noise_real + 1j * noise_imag)
        received_signals += noise
    else:
        raise ValueError("Tipo di canale non valido. Usa 'AWGN' o 'Rayleigh'.")

    averaged_signals = np.mean(received_signals, axis=1)
    X_data = np.concatenate([np.real(averaged_signals), np.imag(averaged_signals)], axis=1)
    return X_data, y_labels

# === BER calculation ===
def calculate_ber(model, beta, m, channel_type='Rayleigh', L=1, min_errors_target=100, max_symbols_limit=10**7, batch_size_for_generation=400000):
    ber_values = []
    snr_points = list(range(0, 21))
    for snr_db in tqdm(snr_points, desc="Calcolo BER", file=sys.stdout):
        errors_count = 0
        total_symbols_tested = 0
        nr_generated_batches = 0
        while errors_count < min_errors_target and total_symbols_tested < max_symbols_limit:
            num_to_generate = min(batch_size_for_generation, max_symbols_limit - total_symbols_tested)
            if num_to_generate <= 0:
                break

            # For reproducibility and guarantee same batches to all models.
            SEED = int.from_bytes(hashlib.sha256(f"{snr_db}-{nr_generated_batches}".encode()).digest()[:4], 'big')

            X_batch_gen, y_true_batch_gen = _generate_test_batch_signals(num_to_generate, snr_db, beta, M, channel_type, L, SEED)
            nr_generated_batches+=1
            predictions = model.predict(X_batch_gen, batch_size=100000, verbose=0)
            predicted_labels = np.argmax(predictions, axis=1)
            current_errors = np.sum(predicted_labels != y_true_batch_gen)
            errors_count += current_errors
            total_symbols_tested += X_batch_gen.shape[0]
        if total_symbols_tested == 0:
            ber = np.nan
        elif errors_count == 0:
            ber = 1.0 / max_symbols_limit
        else:
            ber = errors_count / total_symbols_tested
            print(f"SNR: {snr_db} dB, BER: {ber:.6f} ({errors_count} errori su  {total_symbols_tested} bit)")
        ber_values.append(ber)
    return np.array(ber_values), snr_points

# === Run BER test ===
model_name = "LSTM_DNN_OFDM_DCSK-ORIGINAL-50-BETA_rayleigh_snr_0-20"
model_path = "projects/infocom2026/Rayleigh/trained_models/"+model_name+".h5"
results_folder = "projects/infocom2026/Rayleigh/results"
os.makedirs(results_folder, exist_ok=True)
print(model_file)
lock_folder = "locks"  # dove salveremo i file temporanei dei modelli in esecuzione
os.makedirs(lock_folder, exist_ok=True)

ber_results = {}
snr_points = None

if model_path.endswith(".h5"):
    #model_name = "TDNN_OFDM-DCSK-ORIGINAL-50-BETA_rayleigh_snr_0-20"
    #model_path = os.path.join(cnn_model_folder, model_file)
    result_file = os.path.join(results_folder, f"{model_name}_ber.npy")

    #lock_file = os.path.join(lock_folder, f"{model_name}.lock")

    if os.path.exists(result_file):
        print(f"Risultati già presenti per {model_name}, caricamento...")
        ber_values = np.load(result_file)
    else:
        # Scrivo il file di lock
        #with open(lock_file, "w") as f:
        #    f.write("processing")

        try:
          print(f"Caricamento modello: {model_name}")
          with strategy.scope():
              model = load_model(model_path)
          BETA = 50
          print("\nBeta corrente:"+str(BETA))
          ber_values, snr_points = calculate_ber(model, beta= BETA, m =M)
          np.save(result_file, ber_values)
          print(f"BER salvato in: {result_file}")
          ber_results[model_name] = ber_values
          print(f"BER per {model_name}: {ber_values}")
        finally:
          # Rimuovo il file di lock in ogni caso
          os.remove(lock_file)

Numero di dispositivi in uso dalla strategia: 1
projects/infocom2026/Rayleigh/trained_models/TDNN_OFDM-DCSK-ORIGINAL-50-BETA_rayleigh_snr_0-20.h5
Caricamento modello: TDNN_OFDM-DCSK-ORIGINAL-50-BETA_rayleigh_snr_0-20

Beta corrente:50
Calcolo BER: 100%|██████████████████████████████| 21/21 [04:26<00:00, 12.69s/it]
BER salvato in: projects/infocom2026/Rayleigh/results_otf_test/TDNN_OFDM-DCSK-ORIGINAL-50-BETA_rayleigh_snr_0-20_ber.npy
BER per TDNN_OFDM-DCSK-ORIGINAL-50-BETA_rayleigh_snr_0-20: [0.3988575 0.3652525 0.3260175 0.28119   0.2332225 0.1865925 0.146155
 0.1088075 0.0816275 0.05857   0.041415  0.03011   0.02241   0.01755
 0.0140975 0.0114475 0.0094075 0.0076225 0.0063325 0.00517   0.0043525]
